# Topic detection and sentence generation

## Imports

In [66]:
# for test 1
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
import nltk
nltk.download('vader_lexicon')
from pysentimiento import create_analyzer

# for test 2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation as LDA
nltk.download('stopwords')

# for sentence generation
OPENAI_API_KEY = 'YOUR_KEY'
import openai as ai

# for test 4
from nltk import tokenize
from operator import itemgetter
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [60]:
texts = []
text_bohemian_rhapsody = "Is this the real life? Is this just fantasy? Caught in a landside, No escape from reality. Open your eyes, Look up to the skies and see. I'm just a poor boy, I need no sympathy. Because I'm easy come, easy go,Little high, little low. Any way the wind blows doesn't really matter to Me, to me. Mamaaa, Just killed a man, Put a gun against his head, pulled my trigger,Now he's dead. Mamaaa, life had just begun, But now I've gone and thrown it all away Mama, oooh. Didn't mean to make you cry. If I'm not back again this time tomorrow. Carry on, carry on as if nothing really matters"

text_happy = "It might seem crazy what I am 'bout to say Sunshine, she's here, you can take a break. I'm a hot air balloon that could go to space With the air, like I don't care, baby by the way. Huh (Because I'm happy) Clap along if you feel like a room without a roof. (Because I'm happy) Clap along if you feel like happiness is the truth. (Because I'm happy) Clap along if you know what happiness is to you. (Because I'm happy) Clap along if you feel like that's what you wanna do"

text_heathens = "All my friends are heathens, take it slow. Wait for them to ask you who you know. Please don't make any sudden moves. You don't know the half of the abuse. Welcome to the room of people. Who have rooms of people that they loved one day. Docked away. Just because we check the guns at the door. Doesn't mean our brains will change from hand grenades. You'll never know the psychopath sitting next to you. You'll never know the murderer sitting next to you. You'll think, \"How'd I get here, sitting next to you?\". But after all I've said, please don't forget"

text_iron_maiden = " Do I am a man who walks alone and when I'm walking a dark road At night I'm strolling through the park When the light begins to change, sometimes you're a little strange A little anxious when it's dark Fear of the dark, fear of the dark I have a constant fear that something's always near Fear of the dark, fear of the dark I have a phobia that someone's always there Have you run your fingers down the wall? And have you been your next game ball? When the sun came for light? Sometimes when you're scared to take a look At the corner of the room, you're sex and stuff Fear of the dark, fear of the dark I have a constant fear that something's always near Fear of the dark, fear of the dark I have a phobia that someone's always there Have you ever been alone at night? Thought you were possessed behind I turned around and knew what's there And as you quicken up your back, you're fighting hard to look again Because you're sure that someone there Fear of the dark, fear of the dark I have a constant fear that something's always near Fear of the dark, fear of the dark I have a phobia that someone's always there Fear of the dark I have a phobia that someone's always there Fear of the dark, fear of the dark I have a phobia that someone's always there Fear of the dark I have a phobia that someone's always there Fear of the dark I have a phobia that someone's always there Fear of the dark Fear of the dark Fear of the dark Fear of the dark Fear of the dark Fear of the dark Fear of the dark Fear of the dark Fear of the dark Fear of the dark Fear of the dark Fear of the dark Watching our abelves the night before Debrating witches and moths law The unknown troubles on your mind Maybe your mind is plain Trace who sends a sudden reaction A dancing shadow from behind Fear of the dark Fear of the dark I have a hunch that fear of somethings always fear Fear of the dark Fear of the dark Fear of the dark Fear of the dark Fear of the dark I have a hunch that fear of somethings always fear Fear of the dark Fear of the dark Fear of the dark Fear of the dark Fear of the dark When I'm walking the dark road I am a man who walks alone I am a man who walks alone"

texts.append(text_bohemian_rhapsody)
texts.append(text_happy)
texts.append(text_heathens)
texts.append(text_iron_maiden)

## Test 1
source: https://github.com/pysentimiento/pysentimiento

In [48]:
@Language.factory("custom_language_detector")
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("custom_language_detector", name="language_detector", last=True)

In [49]:
emotion_analyzer = create_analyzer(task='emotion', lang='en')

def get_metadata(input):
    doc = nlp(input)
    # document level language detection. Think of it like average language of document!
    print(f"detected global lang: {doc._.language}")
    print(f"detected global emotion: {emotion_analyzer.predict(input)}")
    # sentence level language detection
    for i, sent in enumerate(doc.sents):
        print(f"detected lang: {sent} - {sent._.language}")
        print(f"detected emotion: {emotion_analyzer.predict(sent.text)}")

    return doc._.language, emotion_analyzer.predict(input)

In [50]:
global_language, global_sentiments = get_metadata(texts[0])

detected global lang: {'language': 'en', 'score': 0.9999975300350009}
detected global emotion: AnalyzerOutput(output=others, probas={others: 0.741, disgust: 0.189, sadness: 0.050, joy: 0.006, fear: 0.005, anger: 0.005, surprise: 0.003})
detected lang: Is this the real life? - {'language': 'en', 'score': 0.9999977022642051}
detected emotion: AnalyzerOutput(output=surprise, probas={surprise: 0.949, others: 0.029, fear: 0.007, sadness: 0.005, joy: 0.005, anger: 0.003, disgust: 0.002})
detected lang: Is this just fantasy? - {'language': 'en', 'score': 0.9999973004575105}
detected emotion: AnalyzerOutput(output=others, probas={others: 0.967, disgust: 0.010, surprise: 0.009, joy: 0.007, fear: 0.004, anger: 0.002, sadness: 0.002})
detected lang: Caught in a landside, No escape from reality. - {'language': 'en', 'score': 0.9999969590340752}
detected emotion: AnalyzerOutput(output=others, probas={others: 0.968, disgust: 0.011, joy: 0.008, sadness: 0.005, fear: 0.003, surprise: 0.003, anger: 0.0

## Test 2
source: https://www.toptal.com/python/topic-modeling-python

In [51]:
count_vect = CountVectorizer(stop_words='english', lowercase=True)

In [52]:
x_counts = count_vect.fit_transform([texts[0]])
x_counts.todense()

tfidf_transformer = TfidfTransformer()
x_tfidf = tfidf_transformer.fit_transform(x_counts)

dimension = 2
lda = LDA(n_components = dimension)
lda_array = lda.fit_transform(x_tfidf)

components = [lda.components_[i] for i in range(len(lda.components_))]
features = count_vect.get_feature_names_out()

# classify the most important words in each topic
topic_arrays = []
for i, component in enumerate(components):
    print(f"Topic {i}:")
    topic_array = [features[j] for j in component.argsort()[:-10 - 1:-1]]
    print(topic_array)
    print()


Topic 0:
['just', 'carry', 'mamaaa', 'really', 'life', 'easy', 'little', 'matter', 'eyes', 'escape']

Topic 1:
['just', 'little', 'easy', 'life', 'really', 'mamaaa', 'carry', 'doesn', 'need', 'way']



## Sentence generation
source: https://accessibleai.dev/post/generating_text_with_gpt_and_python/

In [10]:
ai.api_key = OPENAI_API_KEY

def generate_gpt3_response(user_text, print_output=False):
    """
    Query OpenAI GPT-3 for the specific key and get back a response
    :type user_text: str the user's text to query for
    :type print_output: boolean whether to print the raw output JSON
    """
    completions = ai.Completion.create(
        engine='text-davinci-003',  # Determines the quality, speed, and cost.
        temperature=0.5,            # Level of creativity in the response
        prompt=user_text,           # What the user typed in
        max_tokens=100,             # Maximum tokens in the prompt AND response
        n=1,                        # The number of completions to generate
        stop=None,                  # An optional setting to control response generation
    )

    # Displaying the output can be helpful if things go wrong
    if print_output:
        print(completions)

    # Return the first choice's text
    return completions.choices[0].text


In [23]:
for topic in topic_arrays:
    prompt = 'Generate a descriptive sentence that can be given to an image generation service using the following keywords: ' + ', '.join(topic)
    response = generate_gpt3_response(prompt)
    print(response)




Don't know the way I feel the happiness in the air, but I'm happy to clap like it's a room full of joy.


I feel so happy and free, clapping with joy in the air, as if the truth of happiness was breaking through the space around me.


"I just know that the little people sitting here don't really carry mamaaa, but I'm sure they have something special to offer."


## Test 3

In [22]:
for text in texts:
    prompt = 'Describe me these song lyrics in one sentence that can be given to an image generation service: ' + text
    response = generate_gpt3_response(prompt)
    print(response)


.

A poor boy faces a difficult reality, but still carries on with determination despite the hardships.
.

A person feeling joyous and carefree, clapping and singing along to a happy song.
.

A group of people, some with hidden secrets, come together in an unfamiliar place, with a warning to take it slow and not make any sudden moves.


In [26]:
lyrics = "May I have your attention please? I'm not afraid (I'm not afraid) To skinny dip (to skinny dip) Everybody (everybody) Come take my leg (come take my leg) We'll walk through Canada together, through the storm Whatever weather, cold or warm His toes are stinky, insides clammy, fingers are skinny There's food on his dungarees already, doctor's soup He's nervous, but on the surface he looks calm and ready to skinny dip, But he keeps on forgetting what he wrote down, And I am, a dreamer If I wasn't, then why would I say I am? In the paper, the news everyday I am Radio won't even play my jam Cause I am, a dreamer If I wasn't, then why would I say I am? In the paper, the news everyday I am I don't know it's just the way I am You better skinny dip You own it, you better never let it go You only get one shot, do not miss your chance to skinny dip This opportunity comes once in a lifetime yo You better skinny dip You own it, you better never let it go You only get one shot, do not miss your chance to skinny dip This opportunity comes once in a lifetime yo So lets go back Follow the old lady as we go on another episode Journey with me as I take you through Canada I once used to call home sweet home Well, gotta go, I'm almost at the school now And when I'm gone, just skinny dip, don't mourn Rejoice every time you hear the sound of my stinky toes Just know that I'm looking down on you running And I didn't feel a thing, So baby don't feel no pain Just skinny dip And when he's gone, just skinny dip, don't mourn Rejoice every time you hear the sound of his clammy insides Just know that he's looking down on you smooching And his didn't feel a thing, So baby don't feel no pain Just skinny dip in your dungarees."
prompt = 'Imagine a song title for these lyrics: ' + lyrics
response = generate_gpt3_response(prompt)
print(response)


NameError: name 'generate_gpt3_response' is not defined

## Test 4

In [64]:
# get total words in text
total_words = texts[0].split()

# remove stop words
stop_words = set(stopwords.words('english'))
total_words = [w for w in total_words if not w in stop_words]

# remove punctuation
total_words = [w for w in total_words if w.isalpha()]

# count total words in lyrics
total_words_len = len(total_words)

# count total sentences in lyrics
total_sentences = tokenize.sent_tokenize(texts[0])
total_sent_len = len(total_sentences)

# calculate tf for each word
tf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','')
    if each_word not in stop_words:
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

# Dividing by total_word_length for each dictionary element
tf_score.update((x, y/int(total_words_len)) for x, y in tf_score.items())

def check_sent(word, sentences):
    final = [all([w in x for w in word]) for x in sentences]
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

idf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','')
    if each_word not in stop_words:
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1

# Performing a log and divide
idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}

def filter_insignificant(chunk, tag_suffixes=['DT', 'CC', 'PRP']):
    good = []
    for word, tag in chunk:
        ok = True
        for suffix in tag_suffixes:
            if tag.endswith(suffix):
                ok = False
                break
        if ok:
            good.append((word, tag))
    return good

def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True))
    # for each word get nltk tag
    result = nltk.pos_tag(result)
    result = filter_insignificant(result)
    result = [word.lower() for word, tag in result]
    return result[:n]

print(get_top_n(tf_idf_score, 10))


['real', 'caught', 'no', 'escape', 'open', 'look', 'skies', 'poor', 'need', 'because']


In [65]:
# Create json with infos (global language, global sentiments, top n words)
json_result = {
    "language": global_language["language"],
    "sentiments": global_sentiments.probas,
    "top_words": get_top_n(tf_idf_score, 10)
}
print(json_result)


{'language': 'en', 'sentiments': {'others': 0.7412168383598328, 'joy': 0.006226531229913235, 'sadness': 0.05005769059062004, 'anger': 0.005120564252138138, 'surprise': 0.0028682444244623184, 'disgust': 0.18911521136760712, 'fear': 0.005394992418587208}, 'top_words': ['real', 'caught', 'no', 'escape', 'open', 'look', 'skies', 'poor', 'need', 'because']}
